# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer1.py


## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [3]:
%%bash

hdfs dfs -rm -r -skipTrash assignment1_step1 > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount 1a" \
    -D mapreduce.job.reduces=8 \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output assignment1_step1 > /dev/null 2>yarn1
cat yarn1 >/dev/stderr

19/06/22 14:12:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:12:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:12:48 INFO mapred.FileInputFormat: Total input files to process : 1
19/06/22 14:12:48 INFO mapreduce.JobSubmitter: number of splits:2
19/06/22 14:12:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1561045447920_0042
19/06/22 14:12:49 INFO impl.YarnClientImpl: Submitted application application_1561045447920_0042
19/06/22 14:12:49 INFO mapreduce.Job: The url to track the job: http://0de65cec5c1c:8088/proxy/application_1561045447920_0042/
19/06/22 14:12:49 INFO mapreduce.Job: Running job: job_1561045447920_0042
19/06/22 14:12:54 INFO mapreduce.Job: Job job_1561045447920_0042 running in uber mode : false
19/06/22 14:12:54 INFO mapreduce.Job:  map 0% reduce 0%
19/06/22 14:13:10 INFO mapreduce.Job:  map 32% reduce 0%
19/06/22 14:13:16 INFO mapreduce.Job:  map 44% reduce 0%
19/06/22 14:13:22 INFO 

In [4]:
# %%bash

# hdfs dfs -cat assignment1_step1/part-00000 | head

# Sort task

In [5]:
%%writefile swap1.py

import sys

for line in sys.stdin:
    word, freq = line.strip().split('\t', 1)
    print "%s\t%s" % (freq, word)

Overwriting swap1.py


In [6]:
%%bash

hdfs dfs -rm -r -skipTrash assignment1_step2 > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount 1b" \
    -D mapreduce.job.reduces=1 \
    -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options=-nr \
    -files swap1.py \
    -mapper "python swap1.py" \
    -reducer "python swap1.py" \
    -input assignment1_step1 \
    -output assignment1_step2 > /dev/null 2>yarn2
cat yarn2 >/dev/stderr


rm: `assignment1_step2': No such file or directory
19/06/22 14:13:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:13:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:13:54 INFO mapred.FileInputFormat: Total input files to process : 8
19/06/22 14:13:54 INFO mapreduce.JobSubmitter: number of splits:8
19/06/22 14:13:55 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
19/06/22 14:13:55 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
19/06/22 14:13:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1561045447920_0043
19/06/22 14:13:55 INFO impl.YarnClientImpl: Submitted application application_1561045447920_0043
19/06/22 14:13:55 INFO mapreduce.Job: The url to track the job: http://0de65cec5c1c:8088/proxy/application_1561045447920_0043/


In [7]:
%%bash

cat yarn1 >/dev/stderr
cat yarn2 >/dev/stderr

hdfs dfs -cat assignment1_step2/part-00000 | sed -n '7p;8q'

is	126420


19/06/22 14:12:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:12:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/22 14:12:48 INFO mapred.FileInputFormat: Total input files to process : 1
19/06/22 14:12:48 INFO mapreduce.JobSubmitter: number of splits:2
19/06/22 14:12:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1561045447920_0042
19/06/22 14:12:49 INFO impl.YarnClientImpl: Submitted application application_1561045447920_0042
19/06/22 14:12:49 INFO mapreduce.Job: The url to track the job: http://0de65cec5c1c:8088/proxy/application_1561045447920_0042/
19/06/22 14:12:49 INFO mapreduce.Job: Running job: job_1561045447920_0042
19/06/22 14:12:54 INFO mapreduce.Job: Job job_1561045447920_0042 running in uber mode : false
19/06/22 14:12:54 INFO mapreduce.Job:  map 0% reduce 0%
19/06/22 14:13:10 INFO mapreduce.Job:  map 32% reduce 0%
19/06/22 14:13:16 INFO mapreduce.Job:  map 44% reduce 0%
19/06/22 14:13:22 INFO 